In [1]:
import pandas as pd
import numpy as numpy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from torch.nn.utils.rnn import pad_sequence

import transformers

import sklearn.metrics

In [2]:
train = pd.read_csv('../data/processed/train.csv')
test = pd.read_csv('../data/processed/test.csv')
print('Train: ',train.shape)
print('Test: ',test.shape)

Train:  (67447, 43)
Test:  (18816, 42)


In [15]:
max_len = 60099

class tokenizer:

    def __init__(self):

        self.cls = 0

        self.bases = {
         'A':1,
         'C':2,
         'G':3,
         'N':4,
         'T':5
         }

    def tokenize(self,sequence):

        ids = [self.cls]
        for x in sequence:
            try:
                ids.extend([self.bases[x]])
            except KeyError:
                ('error')

        mask = torch.ones(len(ids))
        ids = torch.Tensor(ids)

        return ids, mask


In [16]:
def getDataset(data):

    sequences = data.sequence.str
    labels = data.target

    input_ids = []
    input_masks = []
    input_token_type = = []
    input_positional = []

    toker = tokenizer()

    for seq in sequences:
        ids, mask = toker.tokenize(seq)
        input_ids.append(ids)
        input_masks.append(mask)
        input_token_type.append(torch.tensor([1]))
        input_positional.append(torch.arange(len(ids)))

    input_ids = pad_sequence(input_ids,batch_first=False,padding_value=0)
    input_masks = pad_sequence(input_masks,batch_first=False,padding_value=0)
    input_token_type = pad_sequence(input_token_type,batch_first=False,padding_value=1)
    input_positional = pad_sequence(input_positional,batch_first=False,padding_value=0)

    return TensorDataset(input_ids,input_masks,input_positional,input_token_type,labels)

In [17]:
train_dataset = getDataset(train)
test_dataset = getDataset(test)

KeyboardInterrupt: 

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    sampler=torch.utils.data.RandomSampler,
    batch_size=8
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=torch.utils.data.SequentialSampler,
    batch_size=8
)

In [48]:
class Net:

    def __init__(self):

        self.bert_config = transformers.BertConfig(vocab_size=6, hidden_size=30, num_hidden_layers=12, num_attention_heads=12, intermediate_size=50, hidden_act='gelu', hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1, max_position_embeddings=60100, type_vocab_size=1, initializer_range=0.02, layer_norm_eps=1e-12, pad_token_id=0,gradient_checkpointing=False, **kwargs)

        self.bert = transformers.BertModel(config)
        self.fc = nn.Linear(768,1314)

    def forward(self):


device = torch.device('cuda')

tensor([[0, 1, 2, 3, 3, 1, 4, 5],
        [0, 1, 1, 5, 0, 0, 0, 0]])

In [ ]:
epochs = 4

optimizer = torch.optim.AdamW(
            model.parameters(),
            lr = float(model_config.lr),
            eps = float(model_config.eps)
        )

total_steps = len(train_dataloader) * epochs

scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                    num_warmup_steps = 0,
                    num_training_steps = total_steps)

for epochs in range(0,epochs):

    logits = []
    total_train_loss = 0

    for step, batch in enumerate(train_dataloader):

            model.zero_grad()

             b_loss, b_logits = model(input_ids=batch[0], attention_mask=batch[1], token_type_ids=batch[3],                         position_ids=batch[2], head_mask=None, inputs_embeds=None, labels=batch[4],                                    output_attentions=False,output_hidden_states=False)

            logits.extend(b_logits)
            ground_truth.extend(b_labels)
                    
            total_train_loss += b_loss.item()

            loss.backward()
                    
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()

            scheduler.step()

        y_probas = nn.Sigmoid()(torch.Tensor(logits))
        y_labels = y_probas.round()

        train_auc = metrics.roc_auc_score(ground_truth,y_probas)
        train_acc = metrics.accuracy_score(ground_truth,y_labels)
        avg_train_loss = total_train_loss/len(train_dataloader)
